<a href="https://colab.research.google.com/github/rohanjain1648/TECHNOVA-HACKATHON/blob/main/rd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# Set seaborn style
sns.set()

# Read dataset
data = pd.read_csv("bs140513_032310.csv")




In [7]:
# Preprocessing
data = data.drop(['zipcodeOri', 'zipMerchant'], axis=1)
col_categorical = data.select_dtypes(include=['object']).columns
for col in col_categorical:
    data[col] = data[col].astype('category').cat.codes

X = data.drop(['fraud'], axis=1)
y = data['fraud']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Model training
knn = KNeighborsClassifier(n_neighbors=5, p=1)
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=8, random_state=42, class_weight="balanced")
xgb_clf = xgb.XGBClassifier(max_depth=6, learning_rate=0.05, n_estimators=400, objective="binary:hinge", random_state=42)

knn.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
xgb_clf.fit(X_train, y_train)

# Ensemble model
ensemble = VotingClassifier(estimators=[("KNN", knn), ("RF", rf_clf), ("XGB", xgb_clf)], voting="soft", weights=[1,4,1])
ensemble.fit(X_train, y_train)

# Real-time input function
def get_user_input():
    user_data = {}
    for column in X.columns:
        value = input(f"Enter value for {column}: ")
        try:
            user_data[column] = float(value)
        except ValueError:
            user_data[column] = 0  # Default if conversion fails
    return pd.DataFrame([user_data])

def predict_real_time(model):
    print("\n--- Real-Time Anomaly Detection ---")
    user_df = get_user_input()
    prediction = model.predict(user_df)[0]
    probability = model.predict_proba(user_df)[0][1]

    print(f"\nAnomaly Detection Result: {'Fraudulent' if prediction == 1 else 'Non-Fraudulent'}")
    print(f"Risk Score (Probability of Fraud): {probability:.4f}")

# Predict on real-time user input
predict_real_time(ensemble)



--- Real-Time Anomaly Detection ---
Enter value for step: 10
Enter value for customer: 300
Enter value for age: 25
Enter value for gender: M
Enter value for merchant: 2000
Enter value for category: 5
Enter value for amount: 10000

Anomaly Detection Result: Fraudulent
Risk Score (Probability of Fraud): 0.8219
